# 🧬 AMES Mutagenicity Prediction

## 📖 Dataset Description
Mutagenicity refers to the ability of a drug to induce genetic alterations. Drugs that damage DNA may lead to cell death or other severe adverse effects.

The **AMES test**, developed by Professor Bruce Ames, is a widely used biological assay to assess mutagenicity. It is a short-term bacterial reverse mutation assay that detects many compounds that can cause genetic damage or frameshift mutations.

The **AMES dataset** used here is aggregated from four scientific studies.

- **Task Type**: Binary classification  
- **Input**: SMILES string of a drug molecule  
- **Output**: `1` (mutagenic) or `0` (not mutagenic)  
- **Size**: 7,255 compounds

## 📚 References
Xu, Congying, et al. “In silico prediction of chemical Ames mutagenicity.” Journal of chemical information and modeling 52.11 (2012): 2840-2847.

**License**: CC BY 4.0

## 📦 Imports

In [1]:
import os
import sys

sys.path.append(os.path.abspath(".."))

import pandas as pd
from scripts.tdc_dataset_download import TDCDatasetDownloader
from scripts.eda_utils import DatasetLoader, EDAVisualizer, SMARTSPatternAnalyzer
from scripts.featurise import MolecularFeaturiser


______

## 📥 Download AMES Dataset

In [2]:
######################################## Declare category and dataset ######################################## 

category = 'tox'
dataset = 'AMES'

In [ ]:
######################################## Initiate downloader class to download the dataset ######################################## 

downloader = TDCDatasetDownloader(category, dataset)

________

## 📊 Exploratory Data Analysis

In this section, we analyze the AMES mutagenicity dataset to better understand its structure, balance, and molecular content. Exploratory Data Analysis (EDA) helps us uncover patterns, detect anomalies, and gain insights that will guide our preprocessing and modeling steps.

### 1. Load Datasets ###

In [ ]:
######################################## Initialize the dataset loader ######################################## 
loader = DatasetLoader(dataset_name=dataset)

In [ ]:
######################################## Load datasets ######################################## 
main_df, train_df, valid_df, test_df = loader.load_all()

______

### 2. 📦 Dataset Overview ###

Before diving into the analysis, it’s essential to understand the structure of the datasets we’re working with — including the main dataset and the train/validation/test splits.

The function `EDAVisualizer.show_dataset_info()` provides a concise summary of:
- Shape (rows × columns)
- Column names
- Missing values
- Sample preview (via `.head()`)

You can run this by:
```python
EDAVisualizer.show_dataset_info(loader)

```
Alternatively, for specific splits only e.g for just Train and Test Data:
```python
selected_datasets = ['train','test']
EDAVisualizer.show_dataset_info(loader, dataset_names=selected_datasets)



In [ ]:
######################################## Show Dataset Info  ########################################

EDAVisualizer.show_dataset_info(loader)

-----

### 3. 💎 Unique SMILES Analysis ###

Knowing how many unique compounds exist in each dataset helps:
- Measure diversity
- Avoid duplication bias
- Confirm splits are stratified

Run:
```python
EDAVisualizer.compare_unique_smiles(
    dfs=[train_df, valid_df, test_df],
    df_names=['Train', 'Valid', 'Test'],
    smiles_col='Drug'  
)


In [ ]:
######################################## Check Unique  Drug  Count in each dataset  ########################################
print(f"Number of Unique drug count in {dataset} train data is: =====>> {train_df["Drug"].nunique()}\n")
print(f"Number of Unique drug count in {dataset} validation data is: =====>> {valid_df["Drug"].nunique()}\n")
print(f"Number of Unique drug count in {dataset} test data is: =====>> {test_df["Drug"].nunique()}\n")
print(f"Total number of Unique drug count in {dataset} data is: =====>> {main_df["Drug"].nunique()}\n")

In [ ]:
######################################## Compare Unique  Drug  Count Distribution  ########################################
EDAVisualizer.compare_unique_smiles(
    dfs=[train_df, valid_df, test_df],
    df_names=['Train', 'Valid', 'Test'],
    smiles_col='Drug'  
)

_____

### 4. 🧮  Target Class Distribution in dataset ###

Class imbalance is a common challenge in classification tasks. Plotting the distribution of the target labels helps us understand:
- Whether the dataset is balanced
- The dominant class (if any)
- The need for resampling or weighted loss functions

We use:
```python
EDAVisualizer.plot_label_distribution(df)


In [ ]:
########################################  Plot class distribution on Main Dataset ######################################## 
EDAVisualizer.plot_label_distribution(main_df, target_col='Y', title='Main - Target Mutagenicity Distribution')

In [ ]:
########################################  Plot class distribution Comparison between Split Dataset ######################################## 
EDAVisualizer.compare_label_distributions(
    dfs=[train_df, valid_df, test_df], 
    df_names=['Train', 'Valid', 'Test'], 
    target_col='Y'
)

______

### 5. 📏 SMILES Length Analysis ###

SMILES strings vary in length depending on the molecular complexity. Analyzing their length:
- Highlights outliers or unusually long/short molecules
- Informs sequence-based model designs (like RNNs, Transformers)

To analyze the distribution:
```python
EDAVisualizer.check_smiles_length(loader=loader)


In [ ]:
########################################  Check the SMILES Length Distribution of the Full Dataset ######################################## 
EDAVisualizer.check_smiles_length(loader=loader)

In [ ]:
########################################  Alternatively, Check the SMILES Length Distribution of the Selected/Multiple Dataset Splits ######################################## 
EDAVisualizer.check_smiles_length(dfs=[test_df], names=["Test"])


In [ ]:
########################################  Compare the SMILES Length Distribution between Split Dataset ######################################## 

EDAVisualizer.compare_smiles_length(loader=loader)

In [ ]:
######################################## Alternatively, we can compare selected splits dataset by executing below ######################################## 

selected_dfs = [train_df, valid_df]
dataset_names=['Train', 'Validation']


EDAVisualizer.compare_smiles_length(selected_dfs, dataset_names)

_____________

### 5. ✔️ RDKit Molecular Validity Check ###

Not all SMILES strings are guaranteed to represent valid molecules. Some may contain syntax errors or rare patterns RDKit cannot parse.

This function evaluates validity by attempting to convert each SMILES to an RDKit Mol object:
```python
EDAVisualizer.check_molecular_validity(loader=loader)


You can also pass a different column name if your SMILES column isn’t named Drug e.g:

EDAVisualizer.check_molecular_validity(train_df, smiles_col='SMILES')

In [ ]:
######################################## Check Drug Molecular Validity ######################################## 

EDAVisualizer.check_molecular_validity(loader=loader)

____________

### 6. 🧬 Molecular Descriptor Engineering ###
Use RDKit to calculate standard drug-likeness properties for molecules — a key step in both Exploratory Data Analysis (EDA) and featurization.

This function computes key cheminformatics descriptors from SMILES using RDKit:
- Molecular weight (MW)
- LogP (lipophilicity)
- Topological Polar Surface Area (TPSA)
- Hydrogen Bond Donors/Acceptors (HBD/HBA)
- Rotatable bonds
- Ring counts (total and aromatic)

These descriptors provide chemical insights into your dataset during EDA (e.g., distribution of molecular weights) and also serve as informative features for downstream machine learning models.

You can add descriptors to a dataset via:
```python
EDAVisualizer.add_molecular_descriptors(loader=loader)

```
You can also:

🔹 Apply to Specific DataFrames:

```python
EDAVisualizer.add_molecular_descriptors(dfs=[train_df, test_df], names=["Train", "Test"])
```

🔹 Apply to a single DataFrame:

```python
EDAVisualizer.add_molecular_descriptors(dfs=valid_df, names=["Validation"])
```

🔹 Return new DataFrames:

```python
updated = EDAVisualizer.add_molecular_descriptors(dfs=[train_df], inplace=False)

In [ ]:
######################################## Add Molecular Descriptors ######################################## 
EDAVisualizer.add_molecular_descriptors(loader=loader)


_____

### 7. ✨ SMARTS Pattern Matching ###

SMARTS patterns represent functional groups (e.g., nitro groups, amines, halogens). This module detects presence of such substructures and summarizes their occurrence by class.

**Steps**:
1. Use `SMARTSPatternAnalyzer().analyze(df)` to add SMARTS flags.
2. Use `.summarize_patterns(df)` to compare frequency by label.

Example:
```python
smarts_analyzer = SMARTSPatternAnalyzer()
train_df = smarts_analyzer.analyze(train_df)
smarts_analyzer.summarize_patterns(train_df)


In [ ]:
######################################## Initialize SMARTS Pattern Analyzer ######################################## 
analyzer = SMARTSPatternAnalyzer()

In [ ]:
######################################## Detect SMARTS Substructures in Main Dataset ######################################## 

data_with_flags = analyzer.analyze(main_df)

In [ ]:
########################################  Analyze SMARTS Substructures ######################################## 

analyzer.summarize_patterns(data_with_flags)

__________

### 8. 📈 Visualize correlation between features  ###

### Feature Correlation Heatmaps

Correlation matrices help identify:
- Redundant features
- Feature interactions
- Potential for multicollinearity

We visualize correlations for numeric descriptors:
```python
EDAVisualizer.compare_correlation_heatmaps(
    [main_df],
    ["Data"],
    cols=['MW', 'LogP', 'TPSA', 'HBD', 'HBA', 'RotBonds', 'RingCount', 'AromaticRings', 'Y']
)
```

You can also visualize for multiple dataset by:
```python
EDAVisualizer.compare_correlation_heatmaps(
    dfs=[train_df, valid_df, test_df],
    df_names=["Train", "Validation", "Test"],
    cols=['MW', 'LogP', 'TPSA', 'HBD', 'HBA', 'RotBonds', 'RingCount', 'AromaticRings']
)


In [ ]:
######################################## Show Correlation Heatmaps for Molecular Descriptors ######################################## 

EDAVisualizer.compare_correlation_heatmaps(
    [main_df],
    ["Data"],
    cols=['MW', 'LogP', 'TPSA', 'HBD', 'HBA', 'RotBonds', 'RingCount', 'AromaticRings', 'Y']
)


In [ ]:
######################################## Check Correlation Heatmaps across Splits ######################################## 

EDAVisualizer.compare_correlation_heatmaps(
    [main_df, valid_df, test_df],
    ["Train", "Validation", "Test"],
    cols=['MW', 'LogP', 'TPSA', 'HBD', 'HBA', 'RotBonds', 'RingCount', 'AromaticRings', 'Y']
)

### Boxplot of Numeric Features Across Splits

Boxplots give another visual cue on median, spread, and outliers per descriptor.

Use:
```python
EDAVisualizer.compare_boxplots(
    dfs=[train_df, valid_df, test_df],
    df_names=['Train', 'Validation', 'Test'],
    col='MW'
)


In [ ]:
######################################## Compare MW Boxplots ######################################## 

EDAVisualizer.compare_boxplots(
    [main_df],
    ["AMES Data"],
    col='MW'
)

In [ ]:
######################################## Compare MW Boxplots ######################################## 

EDAVisualizer.compare_boxplots(
    [train_df, valid_df, test_df],
    ["Train", "Validation", "Test"],
    col='MW'
)

### Compare Feature Distributions (Histogram)

Distribution shifts between train/test datasets can impact model generalization. Here, we compare numeric descriptor histograms.

Example:
```python
EDAVisualizer.compare_numeric_distribution(
    dfs=[train_df, test_df],
    df_names=['Train', 'Test'],
    col='MW'  # or LogP, TPSA, etc.
)


In [ ]:
######################################## xxxxx ######################################## 

EDAVisualizer.compare_numeric_distribution(
    [train_df, valid_df, test_df],
    ["Train", "Validation", "Test"],
    col='LogP'
)

_____________

### 9. 🔍 Visual Inspection of Molecules ###

Before building models, it’s helpful to get a “chemical feel” for the data by visualizing molecules per class.

This function:
- Randomly samples `n` molecules per class
- Converts them to RDKit molecules
- Displays them with grid labels

Usage:
```python
EDAVisualizer.draw_samples_by_class(main_df, n=4)



In [ ]:
######################################## Visualize "n" samples of molecules per class ########################################
EDAVisualizer.draw_samples_by_class(main_df, n=5)

In [ ]:
######################################## Draw Sample DRUG SMILE ########################################

EDAVisualizer.draw_molecules([
    "CC(=O)OC1=CC=CC=C1C(=O)O",  # Aspirin
    "CC(C)CC1=CC=C(C=C1)C(C)C(=O)O"  # Ibuprofen
])

________

## 🧬 Molecular Featurization using Ersilia Representation Model

In this phase, each molecule in the AMES dataset is converted into a numerical vector representation using a pretrained model from the **[Ersilia Model Hub](https://www.ersilia.io/model-hub)**.

The **ErG 2D Descriptors**  representation model was used:

> The Extended Reduced Graph (ErG) approach encodes pharmacophoric properties of molecules by describing their pharmacophore nodes.  
> This model (`eos5guo`) captures the size, shape, and functional characteristics of molecules, making it particularly effective for identifying pharmacophoric patterns.  
> It was benchmarked against Daylight fingerprints and outperformed them in 10 out of 11 tasks.  
> ErG descriptors are especially useful in **scaffold hopping**, a key strategy in drug discovery.

📦 **Model:** `eos5guo`  
🔗 **Source Code:** [github.com/ersilia-os/eos5guo](https://github.com/ersilia-os/eos5guo)  
📖 **Reference Publication:** [10.1021/ci050457y](https://pubs.acs.org/doi/10.1021/ci050457y)

### How the process works:

- The `MolecularFeaturiser` class checks if the model is installed. If not, it **automatically fetches it** using the Ersilia Python SDK.
- The `MolecularFeaturiser` can also be initiated using CLI by setting `use_python_api=False`
- Additionally, by setting `auto_serve=True`, the class tries serving using Python SDK first; but if it fails, switches to CLI mode and logs both events and confirms when initialization is successful.
- It then applies the model to the `train`, `val`, and `test` splits of the AMES dataset.
- This will featurize the train, validation, and test splits.
- Each SMILES string is transformed into a high-dimensional vector representation.
- The output is stored as new CSV files (e.g., `train_eos5axz_features.csv`) inside `data/AMES/splits/`.
- Then the featurization can be executed and preview the results.


In [3]:
######################################## Initialize featurizer ########################################
featuriser = MolecularFeaturiser(dataset_name=dataset, model_id="eos5guo")

2025-04-02 05:34:49,853 - INFO - [API] Initializing Ersilia model: eos5guo


In [4]:
######################################## Run featurization across all dataset splits ######################################## 

featuriser.featurise_all_splits()

2025-04-02 05:35:06,061 - INFO - [+] Featurizing /Users/taiwoadelakin/Documents/Doc/Projects/outreachy-contributions/data/AMES/splits/train.csv -> /Users/taiwoadelakin/Documents/Doc/Projects/outreachy-contributions/data/AMES/splits/train_eos5guo_features.csv
2025-04-02 05:35:18,206 - INFO - Deleted temporary file: /var/folders/5g/d1fz9jw119x57smh9smql2m00000gn/T/tmpqv0vjs2v.csv
2025-04-02 05:35:18,207 - INFO - Deleted temporary file: /var/folders/5g/d1fz9jw119x57smh9smql2m00000gn/T/tmpqv0vjs2v_output.csv
2025-04-02 05:35:18,215 - INFO - Attempting to merge labels from /Users/taiwoadelakin/Documents/Doc/Projects/outreachy-contributions/data/AMES/AMES.csv
2025-04-02 05:35:18,252 - WARNING - Could not merge labels: 'input'
2025-04-02 05:35:18,903 - INFO - Featurization completed in 12.84 seconds for train split.
2025-04-02 05:35:18,904 - INFO - [+] Featurizing /Users/taiwoadelakin/Documents/Doc/Projects/outreachy-contributions/data/AMES/splits/valid.csv -> /Users/taiwoadelakin/Documents/D

In [5]:
######################################## Load and preview featurized dataset among splits ######################################## 

split_paths = {
    split: featuriser.get_featurized_path(split)
    for split in ["train", "valid", "test"]
}


In [ ]:
######################################## Read featurized Split Datasets ######################################## 

df_feat_train = pd.read_csv(split_paths["train"])
df_feat_valid   = pd.read_csv(split_paths["valid"])
df_feat_test  = pd.read_csv(split_paths["test"])

In [ ]:
######################################## Print Shape of Featurized Splits ######################################## 

print("Featurized Train shape:", df_feat_train.shape)
print("Featurized Validation shape:", df_feat_valid.shape)
print("Featurized Test shape:", df_feat_test.shape)


Featurized Train shape: (5094, 318)
Featurized Validation shape: (728, 318)
Featurized Test shape: (1456, 318)


___________